In [1]:
import sys
sys.path.append('/home/mz1482/project/vt-bayesian-opt-bopt_debug/')
import pandas as pd
import numpy as np
from data_analysis import *
from graph import *
from BayesOptLib.bayes_opt.bayesian_optimization import BayesianOptimization
from RandomSampler import RandomSampler
import matplotlib
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics.pairwise import euclidean_distances
matplotlib.use('Qt5Agg')  # or can use 'TkAgg', whatever you have/prefer
from prettytable import PrettyTable

def black_box(x, y, z):
    """
    Represents a black box function to maximize for CC of two ECGs given an XYZ coordinate
    :param x:
    :param y: coordinates of the prediction
    :param z:
    :return: CC of that point and the target
    """
    sample_ecg = ecgs[get_index(np.array([x, y, z]),labels)]
    return abs(correlation_coef(target_ecg, sample_ecg))


def black_box_12_lead(x, y, z):
    """
    Represents a black box function to maximize for CC of two ECGs given an XYZ coordinate
    :param x:
    :param y: coordinates of the prediction
    :param z:
    :return: CC of that point and the target
    """
    sample_ecg = ecgs[get_index(np.array([x, y, z]),labels)]
    sample_ecg = np.reshape(sample_ecg, [12, -1])
    nums = 0
    for i in range(12):
#         ccs = np.corrcoef(sample_ecg[i], target_ecg.reshape(12,-1)[i])[0, 1]
        ccs = abs(correlation_coef(target_ecg.reshape(12,-1)[i], sample_ecg[i]))
        if ccs > .9:
            nums += 1

#     return True if nums == 12 else False 
    return nums

In [2]:
data_path = "/home/mz1482/project/vt-bayesian-opt-bopt_debug/data/simu_data_4000/"
# Reading in the ECGs and labels
# aucs = pd.read_csv(data_path+"Heart1/Heart1_AUCS.csv", header=None).to_numpy()
ecgs = pd.read_csv(data_path+"Heart1/Heart1_SimuData_4000.csv", header=None).to_numpy()
labels = pd.read_csv(data_path+"Heart1/Coord1_4000.csv", header=None).to_numpy() / 1000
uvc = pd.read_csv(data_path+"UVC/Coord1_UVC_4000.csv", header=None).to_numpy()
bounds = get_heart_bounds(labels) # getting bounds 

# print("Target: ", target)
init = 10 #initial random sample
steps = 20 # total AL step
af = "ucb" 


In [3]:
# labels = uvc[:,0:3]
# bounds = get_heart_bounds(labels) # getting bounds 
# print("Target: ", target_uvc)
for n in range(3):
    tidx = np.random.randint(0, labels.shape[0])    # Pick out a sample to use as a target
    target, target_ecg,target_uvc = labels[tidx], ecgs[tidx], uvc[tidx]
    optimizer = BayesianOptimization(f=black_box,pbounds=bounds,random_state=None, real_set=labels)
    gp,X = optimizer.gpfit(init_points=init, n_iter=steps,  acq=af, kappa = 2.5,kappa_decay=0.90,kappa_decay_delay=5,xi=0)
    #     heart(target,labels)
    max_cc, al,pass_lead,loc = result_metric(optimizer,target,target_ecg,labels,ecgs)
    f= open("result.txt","a")
    f.write(str(n)+","+str(target[0]) + "," + str(target[1]) + "," + str(target[2])+","+str(max_cc)+","+str(al)+","+str(pass_lead)+","+str(loc) +"\n")
    f.close()



|   iter    |  target   |     x     |     y     |     z     |
-------------------------------------------------------------
|  1        |  0.1762   | -23.62    | -24.66    | -23.6     |
|  2        |  0.7994   | -40.27    | -90.4     |  17.78    |
|  3        |  0.4583   |  11.85    | -30.2     |  29.6     |
|  4        |  0.2063   | -32.66    | -71.03    |  41.9     |
|  5        |  0.1951   | -26.32    | -55.1     |  27.45    |
|  6        |  0.241    | -22.97    | -50.0     |  23.31    |
|  7        |  0.02181  |  41.92    | -71.93    | -27.22    |
|  8        |  0.283    | -19.75    | -48.77    |  19.17    |
|  9        |  0.3432   |  3.011    | -24.5     |  26.28    |
|  10       |  0.2202   | -23.78    | -50.73    |  24.85    |
|  11       |  0.9192   | -45.44    | -97.56    |  6.307    |
|  12       |  0.9426   | -46.94    | -98.08    |  6.165    |
|  13       |  0.9939   | -49.09    | -102.5    | -13.0     |
best value [ -49.0911 -102.545   -12.9966]
|   iter    |  target   |  

In [9]:
data = np.loadtxt("result.txt", delimiter=",")

26.7018

In [4]:
i=1

In [5]:
max_cc

1.0

In [6]:
    init = len(optimizer.visited)-len(optimizer.predicted)
    Y = optimizer._space.target
    X = optimizer.visited
    max_cc = np.amax(Y)
    for i in range(len(Y)):
        if Y[i]==max_cc:
            best = X[i]
            break
    print("best value",best.shape)

best value (3,)


In [7]:


def result_table_xyz(target,optimizer,uvc=False):
    '''
    for 1 experiment it prints/return the table with distance from target at every iteration.
    This function is for xyz coordinates
    '''
    print("The target location (in 3d) is:",target)
    if uvc==False:
        visited = np.asarray(optimizer.visited)
        f = optimizer._space.target
        nn_dis = np.empty((0,1))
        for i in range(len(visited)):
            d = np.sqrt(np.sum((target - visited[i])**2))
            nn_dis = np.append(nn_dis,d)
        nn_dis = nn_dis.reshape(-1,1)
        nn_dis = np.around(nn_dis,2)
        f = f.reshape(-1,1).astype(int)
        it = (np.arange(len(visited))+1).reshape(-1,1).astype(int)
        floating_part = np.around(np.concatenate((visited,nn_dis), axis = 1),2)
        int_part = np.concatenate((it,f), axis = 1)
        result = np.concatenate((int_part,floating_part), axis = 1)
        table = PrettyTable(result.dtype.names)
        table.field_names = ['iteration','passing_lead', 'x','y','z','dis(mm) from target']
        for row in result:
            table.add_row(row)
        return table
    
    

In [8]:
result_table_xyz(target,optimizer)

The target location (in 3d) is: [ -8.8329 -38.2192  26.7002]


ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [ ]:
table.title = 'Results for method Foo'
table.field_names = ['Experiment', 'Value']
table.add_row(['bla', 3.14])
table.add_row(['baz', 42.0])

In [ ]:
len(optimizer.predicted)

In [ ]:
f.astype(int)

In [ ]:
it.shape